In [3]:
from datetime import datetime, date
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder.getOrCreate()

22/07/13 08:54:05 WARN Utils: Your hostname, not resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface enp7s0)
22/07/13 08:54:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/13 08:54:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
datf=spark.createDataFrame([
    Row(Cidade='Salvador', Populacao=6., Time='Bahia', Dat_cadastro=date(2021, 10, 10)),
    Row(Cidade='Sao Paulo',Populacao=45., Time='Sao Paulo', Dat_cadastro=date(2021, 5, 5)),
    Row(Cidade='Recife', Populacao=3., Time='Sport',Dat_cadastro=date(2021, 7, 2)),
    Row(Cidade='Maceio', Populacao=2., Time='CSA', Dat_cadastro=date(2021, 8, 1)),
])

datf

DataFrame[Cidade: string, Populacao: double, Time: string, Dat_cadastro: date]

In [5]:
datf.show()

+---------+---------+---------+------------+
|   Cidade|Populacao|     Time|Dat_cadastro|
+---------+---------+---------+------------+
| Salvador|      6.0|    Bahia|  2021-10-10|
|Sao Paulo|     45.0|Sao Paulo|  2021-05-05|
|   Recife|      3.0|    Sport|  2021-07-02|
|   Maceio|      2.0|      CSA|  2021-08-01|
+---------+---------+---------+------------+



In [6]:
datf.show(2)

+---------+---------+---------+------------+
|   Cidade|Populacao|     Time|Dat_cadastro|
+---------+---------+---------+------------+
| Salvador|      6.0|    Bahia|  2021-10-10|
|Sao Paulo|     45.0|Sao Paulo|  2021-05-05|
+---------+---------+---------+------------+
only showing top 2 rows



In [7]:
datf.show(3, vertical=True)

-RECORD 0------------------
 Cidade       | Salvador   
 Populacao    | 6.0        
 Time         | Bahia      
 Dat_cadastro | 2021-10-10 
-RECORD 1------------------
 Cidade       | Sao Paulo  
 Populacao    | 45.0       
 Time         | Sao Paulo  
 Dat_cadastro | 2021-05-05 
-RECORD 2------------------
 Cidade       | Recife     
 Populacao    | 3.0        
 Time         | Sport      
 Dat_cadastro | 2021-07-02 
only showing top 3 rows



In [8]:
datf.collect()

[Row(Cidade='Salvador', Populacao=6.0, Time='Bahia', Dat_cadastro=datetime.date(2021, 10, 10)),
 Row(Cidade='Sao Paulo', Populacao=45.0, Time='Sao Paulo', Dat_cadastro=datetime.date(2021, 5, 5)),
 Row(Cidade='Recife', Populacao=3.0, Time='Sport', Dat_cadastro=datetime.date(2021, 7, 2)),
 Row(Cidade='Maceio', Populacao=2.0, Time='CSA', Dat_cadastro=datetime.date(2021, 8, 1))]

In [9]:
datf.printSchema()

root
 |-- Cidade: string (nullable = true)
 |-- Populacao: double (nullable = true)
 |-- Time: string (nullable = true)
 |-- Dat_cadastro: date (nullable = true)



In [10]:
datf.select("Cidade", "Populacao").describe().show()

+-------+---------+-----------------+
|summary|   Cidade|        Populacao|
+-------+---------+-----------------+
|  count|        4|                4|
|   mean|     null|             14.0|
| stddev|     null|20.73644135332772|
|    min|   Maceio|              2.0|
|    max|Sao Paulo|             45.0|
+-------+---------+-----------------+



In [11]:
from pyspark.sql.functions import upper
datf.withColumn('Cidade_U', upper(datf.Cidade)).show()

+---------+---------+---------+------------+---------+
|   Cidade|Populacao|     Time|Dat_cadastro| Cidade_U|
+---------+---------+---------+------------+---------+
| Salvador|      6.0|    Bahia|  2021-10-10| SALVADOR|
|Sao Paulo|     45.0|Sao Paulo|  2021-05-05|SAO PAULO|
|   Recife|      3.0|    Sport|  2021-07-02|   RECIFE|
|   Maceio|      2.0|      CSA|  2021-08-01|   MACEIO|
+---------+---------+---------+------------+---------+



In [13]:
datf.filter(datf.Cidade == "Salvador").show()

+--------+---------+-----+------------+
|  Cidade|Populacao| Time|Dat_cadastro|
+--------+---------+-----+------------+
|Salvador|      6.0|Bahia|  2021-10-10|
+--------+---------+-----+------------+



In [14]:
datf.createOrReplaceTempView("Dados")
spark.sql("select count(*) from Dados").show()

+--------+
|count(1)|
+--------+
|       4|
+--------+



In [15]:
# carregando os dados sobre Capital de paises
pais = [("Brasil",10000),("Argentina",20000),("Australia",35000),("Italia",40000),("Egito",65000),("Mexico",80000)]
rddpais= spark.sparkContext.parallelize(pais)

In [16]:
dataframerdd = rddpais.toDF()

In [17]:
dataframerdd.show()

+---------+-----+
|       _1|   _2|
+---------+-----+
|   Brasil|10000|
|Argentina|20000|
|Australia|35000|
|   Italia|40000|
|    Egito|65000|
|   Mexico|80000|
+---------+-----+



In [19]:
colunas = ['Pais', 'Total_capital(Bilhoes)']
dataframerdd2 = rddpais.toDF(colunas)
dataframerdd2.printSchema()
dataframerdd2.show(truncate=False)

root
 |-- Pais: string (nullable = true)
 |-- Total_capital(Bilhoes): long (nullable = true)

+---------+----------------------+
|Pais     |Total_capital(Bilhoes)|
+---------+----------------------+
|Brasil   |10000                 |
|Argentina|20000                 |
|Australia|35000                 |
|Italia   |40000                 |
|Egito    |65000                 |
|Mexico   |80000                 |
+---------+----------------------+

